## 論文実装
https://publicacoes.softaliza.com.br/cilamce/article/view/8202

A Multi-Objective Ant Colony Optimization for Routing in Printed Circuit Boards


In [ ]:
import sys
import math
from heapq import *
from collections import defaultdict
import random
import networkx as nx
import matplotlib.pyplot as plt
from typing import List
sys.setrecursionlimit(10**7)

In [ ]:
n=10

---
### グラフの作成

In [ ]:
n = 10

# 隣接リストの作成
to = [[] for _ in range(n*n)]
for i in range(n*n):
    if (i+1) % n != 0:
        to[i].append(i+1)
    if i % n != 0:
        to[i].append(i-1)
    if i+n < n*n:
        to[i].append(i+n)
    if i-n >= 0:
        to[i].append(i-n)

In [ ]:
# グラフの可視化
def vis_gridpath(n, path_nodes, obs_nodes=None):
    # n×n の格子グラフを作成
    G = nx.grid_2d_graph(n, n)
    mapping = {(i, j): i*n + j for i, j in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
    path_edges = list(zip(path_nodes[:-1], path_nodes[1:]))

    # 描画
    plt.figure(figsize=(6,6))

    # 通常ノード・エッジ
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=500)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_labels(G, pos)

    # 障害物エリアの描画
    if obs_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=obs_nodes, node_color='black', node_size=500)
        labels = {n: n for n in obs_nodes}
        nx.draw_networkx_labels(
            G, pos,
            labels=labels,
            font_color='white',
        )

    # 経路部分を強調（色を変える）
    nx.draw_networkx_nodes(G, pos, nodelist=path_nodes, node_color='orange')
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

    plt.axis('off')
    plt.show()

---
## 論文実装



In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random

In [ ]:
n = 10

In [ ]:
# --- グラフ構造の定義（2次元格子グラフ） ---
n = 10  # グリッドのサイズ（n×n）
to = [[] for _ in range(n * n)]  # 隣接リスト
for i in range(n * n):
    if (i + 1) % n != 0:
        to[i].append(i + 1)
    if i % n != 0:
        to[i].append(i - 1)
    if i + n < n * n:
        to[i].append(i + n)
    if i - n >= 0:
        to[i].append(i - n)

In [ ]:
# # --- 経路の可視化関数 ---
# def vis_gridpath(n, path_nodes, obs_nodes=None):
#     G = nx.grid_2d_graph(n, n)
#     mapping = {(i, j): i * n + j for i, j in G.nodes()}
#     G = nx.relabel_nodes(G, mapping)
#     pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
#     path_edges = list(zip(path_nodes[:-1], path_nodes[1:]))

#     plt.figure(figsize=(6, 6))
#     nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=500)
#     nx.draw_networkx_edges(G, pos, edge_color='gray')
#     nx.draw_networkx_labels(G, pos)

#     if obs_nodes:
#         nx.draw_networkx_nodes(G, pos, nodelist=obs_nodes, node_color='black', node_size=500)
#         labels = {n: n for n in obs_nodes}
#         nx.draw_networkx_labels(G, pos, labels=labels, font_color='white')

#     nx.draw_networkx_nodes(G, pos, nodelist=path_nodes, node_color='orange')
#     nx.draw_networkx_edges(G, pos, edgelist=path_edges, edge_color='red', width=2)

#     plt.axis('off')
#     plt.show()


def vis_gridpath(n, paths, obs_nodes=None):
    """
    グリッドグラフ上に経路を描画する。

    Args:
        n (int):
        paths (List[List[int]]): 経路の集合。一つの経路はnodeのwaypointから構成される。
    Returns:
        None
    """
    node_colors = ['orange', 'cyan', 'greenyellow', 'violet']
    edge_colors = ['red', 'blue', 'green', 'blueviolet']
    G = nx.grid_2d_graph(n, n)
    mapping = {(i, j): i * n + j for i, j in G.nodes()}
    G = nx.relabel_nodes(G, mapping)
    pos = {i * n + j: (j, -i) for i in range(n) for j in range(n)}
    plt.figure(figsize=(6, 6))
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=500)
    nx.draw_networkx_edges(G, pos, edge_color='gray')
    nx.draw_networkx_labels(G, pos)

    if obs_nodes:
        nx.draw_networkx_nodes(G, pos, nodelist=obs_nodes, node_color='black', node_size=500)
        labels = {n: n for n in obs_nodes}
        nx.draw_networkx_labels(G, pos, labels=labels, font_color='white')

    for (i, path) in enumerate(paths):
        edges = list(zip(path[:-1], path[1:]))
        nx.draw_networkx_nodes(G, pos, nodelist=path, node_color='orange')
        nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color='red', width=2)

        nx.draw_networkx_nodes(G, pos, nodelist=path, node_color=node_colors[i % len(paths)])
        nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=edge_colors[i % len(paths)], width=2)

    plt.axis('off')
    plt.show()


In [ ]:
# --- 評価関数（3目的の加重平均）---
def evaluate_path(path, best_paths, w1=10, w2=45, w3=45):
    length = len(path)
    # 他の経路と交差したノード数
    cross_count = sum(1 for node in path for other in best_paths if other and node in other)
    # 他の経路との長さ差の2乗の合計（長さ整合）
    length_diff = sum((length - len(p)) ** 2 for p in best_paths if p)
    # 加重平均としてスコアを返す
    score = (w1 * length + w2 * cross_count + w3 * length_diff) / (w1 + w2 + w3)
    return score

In [ ]:
# --- 経路生成関数（ACO風ランダム探索） ---
def generate_path(start, goal, pheromone, occupied, alpha=2, beta=2):
    current = start
    path = [current]
    visited = set([current])

    while current != goal:
        neighbors = [v for v in to[current] if v not in visited]
        if not neighbors:
            return []  # 行き止まり

        desirability = []
        for node in neighbors:
            tau = pheromone[node] ** alpha
            eta = 1 / (1 + occupied[node])  # 他配線と重なってないほど好ましい
            desirability.append(tau * (eta ** beta))

        total = sum(desirability)
        probs = [d / total for d in desirability]
        current = random.choices(neighbors, weights=probs)[0]
        path.append(current)
        visited.add(current)

    return path

In [ ]:
# --- ACOによる等長・非交差ルーティング本体 ---
def equal_length_routing(pairs, max_iter=100, num_ants=30):
    # pheromones = [np.ones(n * n) * 0.1 for _ in pairs]  # 初期フェロモン
    pheromones = [np.ones(n * n * n) * 0.1 for _ in pairs]  # 初期フェロモン
    best_paths = [None] * len(pairs)  # 各ペアの最良経路

    for iteration in range(max_iter):
        for i, (start, goal) in enumerate(pairs):
            paths = []
            for _ in range(num_ants):
                # 他のトレースが使っているセルを記録（混雑度）
                # occupied = np.zeros(n * n)
                occupied = np.zeros(n * n * n)
                for j, other_path in enumerate(best_paths):
                    if i != j and other_path:
                        for node in other_path:
                            occupied[node] += 1

                # 経路生成 → 評価
                path = generate_path(start, goal, pheromones[i], occupied)
                if path:
                    # score = evaluate_path(path, best_paths)
                    score = evaluate_path(path, best_paths, w1=3, w2=5, w3=10)
                    paths.append((score, path))

            # 最良経路を採用・フェロモン更新
            if paths:
                best = min(paths, key=lambda x: x[0])[1]
                best_paths[i] = best
                for node in best:
                    pheromones[i][node] += 1.0 / len(best)
                pheromones[i] *= 0.9  # フェロモン蒸発

        # 等長化判定 → 成功したら早期停止
        lengths = [len(p) for p in best_paths if p]
        if len(lengths) == len(pairs) and len(set(lengths)) == 1:
            return best_paths

    return best_paths

---
## main(2-dim)

In [ ]:
# --- 実行部：ペア指定と可視化 ---
# random.seed(42)

random.seed(0)
# random.seed(1)
# random.seed(2)


# pairs = [(0, 99), (9, 90), (10, 89)]  # スタート・ゴールの組

# pairs = [(0, 99), (10, 89), (20, 79)] # 実験1
# npairsets = [(0, 35), (14, 74), (83, 67)] # 実験2
# pairs = [(42, 7), (89, 13), (66, 21)] # 実験3
pairs = [(48, 63), (94, 58), (0, 51)] # 実験4


paths = equal_length_routing(pairs)

# 結果の表示(1本ずつの表示)
for path in paths:
    print('len(path) =', len(path))
    # vis_gridpath(n, path)
    vis_gridpath(n, [path])

# 結果の表示(3本まとめて表示)
vis_gridpath(n, paths)

In [ ]:
# 交差数の計算
from collections import defaultdict

crossdict = defaultdict(int)
for path in paths:
    for p in path:
        crossdict[p] += 1


n_cross = sum(crossdict.values()) - len(crossdict.keys())
print('交差数:', n_cross)

In [ ]:
len(crossdict.keys())

In [ ]:
sum(crossdict.values())

---
## main(3-dim)

In [ ]:
# # --- グラフ構造の定義（3次元格子グラフ） ---
# n = 15  # グリッドのサイズ（n×n）
# to = [[] for _ in range(n * n)]  # 隣接リスト
# for i in range(n * n):
#     if (i + 1) % n != 0:
#         to[i].append(i + 1)
#     if i % n != 0:
#         to[i].append(i - 1)
#     if i + n < n * n:
#         to[i].append(i + n)
#     if i - n >= 0:
#         to[i].append(i - n)

In [ ]:
def coord_to_nodenum(grid_size, x, y, z):
    nodenum = grid_size*grid_size*z + grid_size*y + x
    return nodenum


def nodenum_to_coord(grid_size, nodenum):
    z = nodenum // (grid_size * grid_size)
    rem = nodenum % (grid_size * grid_size)
    y = rem // grid_size
    x = rem % grid_size
    return x, y, z

# 確認(OK)
# print(coord_to_nodenum(5, 0, 0, 1))
print(nodenum_to_coord(5, 11))


In [ ]:
import networkx as nx
import plotly.graph_objects as go

def vis_gridpath_3d(n, paths=None, obs_nodes=None):
    """
    3次元格子グラフ上に経路を描画する（ノード中央ラベル & 表示切替ボタン）
    - ラベルは scene.annotations で完全中央に重ね表示
    - 濃色ノード上の文字は白、灰色ノード上は濃紺
    - n に応じて描画範囲と図サイズを自動スケーリング
    - 灰色ノードは半透明（0.6）
    """
    # --- ノード定義（z層ごと連番） ---
    pos = {}
    counter = 0
    for z in range(n):
        for y in range(n):
            for x in range(n):
                pos[counter] = (x, y, z)
                counter += 1

    # --- エッジ定義（6近傍） ---
    G = nx.Graph()
    for i in range(n):
        for j in range(n):
            for k in range(n):
                idx = k * n * n + j * n + i
                for dx, dy, dz in [(1,0,0), (0,1,0), (0,0,1)]:
                    ni, nj, nk = i + dx, j + dy, k + dz
                    if ni < n and nj < n and nk < n:
                        nidx = nk * n * n + nj * n + ni
                        G.add_edge(idx, nidx)

    # --- 座標 ---
    node_x = [pos[i][0] for i in pos]
    node_y = [pos[i][1] for i in pos]
    node_z = [pos[i][2] for i in pos]

    # --- エッジ座標 ---
    edge_x, edge_y, edge_z = [], [], []
    for e in G.edges():
        x0, y0, z0 = pos[e[0]]
        x1, y1, z1 = pos[e[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
        edge_z += [z0, z1, None]

    # --- エッジ描画 ---
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='gray', width=2),
        hoverinfo='none',
        name='Edges'
    )

    # --- ノードカラー決定 ---
    node_color = ['lightgray'] * (n**3)
    if obs_nodes:
        for obs in obs_nodes:
            if 0 <= obs < len(node_color):
                node_color[obs] = 'black'

    path_colors = ['red', 'blue', 'green', 'orange', 'violet', 'cyan']
    if paths:
        for i, path in enumerate(paths):
            col = path_colors[i % len(path_colors)]
            for node in path:
                node_color[node] = col

    # --- 灰色ノード（半透明）とその他ノード（不透明）を分けて描画 ---
    gray_nodes  = [i for i,c in enumerate(node_color) if c == 'lightgray']
    color_nodes = [i for i,c in enumerate(node_color) if c != 'lightgray']

    traces = [edge_trace]

    if gray_nodes:
        traces.append(go.Scatter3d(
            x=[node_x[i] for i in gray_nodes],
            y=[node_y[i] for i in gray_nodes],
            z=[node_z[i] for i in gray_nodes],
            mode='markers',
            marker=dict(size=9, color='lightgray', opacity=0.6),
            name='Normal Nodes',
            hovertext=[f'Node {i}' for i in gray_nodes],
            hoverinfo='text'
        ))

    if color_nodes:
        traces.append(go.Scatter3d(
            x=[node_x[i] for i in color_nodes],
            y=[node_y[i] for i in color_nodes],
            z=[node_z[i] for i in color_nodes],
            mode='markers',
            marker=dict(size=9, color=[node_color[i] for i in color_nodes], opacity=0.9),
            name='Colored Nodes',
            hovertext=[f'Node {i}' for i in color_nodes],
            hoverinfo='text'
        ))

    # --- 経路 ---
    if paths:
        for i, path in enumerate(paths):
            px, py, pz = [], [], []
            for node in path:
                x, y, z = pos[node]
                px.append(x); py.append(y); pz.append(z)
            traces.append(go.Scatter3d(
                x=px, y=py, z=pz,
                mode='lines+markers',
                line=dict(color=path_colors[i % len(path_colors)], width=6),
                marker=dict(size=10, color=path_colors[i % len(path_colors)]),
                name=f'Path {i}'
            ))

    # --- ノード番号（scene.annotations で完全中央に重ねる） ---
    def make_label_annotations():
        anns = []
        for i in range(len(node_x)):
            c = node_color[i]
            text_color = 'white' if c in ['red','blue','green','orange','violet','cyan','black'] else 'darkblue'
            anns.append(dict(
                showarrow=False,
                x=node_x[i], y=node_y[i], z=node_z[i],
                text=str(i),
                xanchor='center', yanchor='middle',
                font=dict(color=text_color, size=11),
                bgcolor='rgba(0,0,0,0)',  # 背景なし（必要なら半透明背景も可）
                bordercolor='rgba(0,0,0,0)'
            ))
        return anns

    annotations_on  = make_label_annotations()
    annotations_off = []  # 非表示時

    # --- スケーリング設定 ---
    margin = 0.5
    axis_range = [-margin, n - 1 + margin]
    fig_size = max(600, min(1400, 100 + n * 100))

    fig = go.Figure(data=traces)
    fig.update_layout(
        title=f"3D Grid Graph (n={n})",
        showlegend=True,
        legend=dict(itemsizing='constant'),
        scene=dict(
            xaxis=dict(title='X', range=axis_range, showbackground=True),
            yaxis=dict(title='Y', range=axis_range, showbackground=True),
            zaxis=dict(title='Z', range=axis_range, showbackground=True),
            aspectmode='cube',
            annotations=annotations_off  # 初期状態：表示
        ),
        width=fig_size,
        height=fig_size,
        margin=dict(l=0, r=0, b=0, t=30),
        updatemenus=[
            dict(
                type="buttons",
                direction="right",
                x=0.5, y=1.08, xanchor="center", yanchor="bottom",
                buttons=[
                    dict(
                        label="Hide Labels",
                        method="relayout",
                        args=[{"scene.annotations": annotations_off}]
                    ),
                    dict(
                        label="Show Labels",
                        method="relayout",
                        args=[{"scene.annotations": annotations_on}]
                    ),
                ]
            )
        ]
    )

    fig.show()


In [ ]:
import random

def generate_obstacle_nodes_3d(grid_size, obs_size=3, is_random=False):
    """
    3次元的な障害物ノードを作る。障害物の大きさは3*3*3に固定する。

    Args:
        grid_size (int):
        is_random (bool):
    Returns:
        obs (list[int]): 障害物ノードのリスト
    """
    if grid_size <= 3: assert False, "グリッドサイズが小さすぎます。"
    obs = []

    if is_random:
        basepoint = [random.randint(0, grid_size**3-1) for _ in range(n_obs)]
    else:
        # basepoint = [445]
        # basepoint = [11]
        # basepoint = [6]
        # basepoint = [16]
        basepoint = [6, 11]

    n_obs = len(basepoint)
    for i in range(n_obs):
        bp = basepoint[i]

        # 障害物ノードが連結になるかの確認
        bp_x, bp_y, bp_z = nodenum_to_coord(grid_size, bp)
        print(f'(bp_x, bp_y, bp_z) = ({bp_x}, {bp_y}, {bp_z})')
        cond = (grid_size-bp_x >= obs_size) & (grid_size-bp_y >= obs_size) & (grid_size-bp_z >= obs_size)        
        if not cond: continue
        for layer in range(obs_size):
            for row in range(obs_size):
                for col in range(obs_size):
                    add_point = bp + layer*grid_size*grid_size + row*grid_size + col
                    print('add_point =', add_point)
                    obs.append(add_point)

        # 重複ノードを除外する。
        obs = list(set(obs))
    return obs


In [ ]:
obs = generate_obstacle_nodes_3d(grid_size=10)
print(obs)
print('len(obs) =', len(obs))

In [ ]:
# 3x3x3 のグリッドにおける例
paths = [
    [0, 1, 2, 11, 20, 21, 22],
    [5, 14, 23, 24, 25],
    [8, 7, 6],
]

# vis_gridpath_3d(4, paths=paths, obs_nodes=obs)
vis_gridpath_3d(n=10, obs_nodes=obs)

In [ ]:
# --- グラフ構造の定義（3次元格子グラフ） ---
n = 10  # 各軸方向のサイズ（n×n×n）
to_without_obs = [[] for _ in range(n * n * n)]  # 隣接リスト

for i in range(n * n * n):
    # nodenum → (x, y, z) に変換
    z = i // (n * n)
    rem = i % (n * n)
    y = rem // n
    x = rem % n

    # +x 方向
    if x + 1 < n:
        to_without_obs[i].append(i + 1)
    # -x 方向
    if x - 1 >= 0:
        to_without_obs[i].append(i - 1)
    # +y 方向
    if y + 1 < n:
        to_without_obs[i].append(i + n)
    # -y 方向
    if y - 1 >= 0:
        to_without_obs[i].append(i - n)
    # +z 方向
    if z + 1 < n:
        to_without_obs[i].append(i + n * n)
    # -z 方向
    if z - 1 >= 0:
        to_without_obs[i].append(i - n * n)

In [ ]:
for i in range(n*n*n):
    print(f"to_without_obs[{i}] = {to_without_obs[i]}")

In [ ]:
print(obs)

In [ ]:
# --- グラフ構造の定義（3次元格子グラフ） ---
n = 10  # 各軸方向のサイズ（n×n×n）
to = [[] for _ in range(n * n * n)]  # 隣接リスト

for i in range(n * n * n):
    # nodenum → (x, y, z) に変換
    z = i // (n * n)
    rem = i % (n * n)
    y = rem // n
    x = rem % n

    # +x 方向
    if x + 1 < n:
        if not i+1 in obs:
            to[i].append(i + 1)
    # -x 方向
    if x - 1 >= 0:
        if not i-1 in obs:
            to[i].append(i - 1)
    # +y 方向
    if y + 1 < n:
        if not i+n in obs:
            to[i].append(i + n)
    # -y 方向
    if y - 1 >= 0:
        if not i-n in obs:
            to[i].append(i - n)
    # +z 方向
    if z + 1 < n:
        if not i + n*n in obs:
            to[i].append(i + n * n)
    # -z 方向
    if z - 1 >= 0:
        if not i - n*n in obs:
            to[i].append(i - n * n)

In [ ]:
for i in range(n*n*n):
    print(f"to[{i}] = {to[i]}")

In [ ]:
# --- 実行部：ペア指定と可視化 ---
random.seed(0)
# random.seed(1)
# random.seed(2)


pairs = [(1, 19), (4, 140), (15, 452)] # 実験


paths = equal_length_routing(pairs)

# 結果の表示(1本ずつの表示)
for path in paths:
    print('len(path) =', len(path))
    # vis_gridpath(n, path)
    vis_gridpath_3d(n, [path], obs_nodes=obs)

# 結果の表示(3本まとめて表示)
vis_gridpath_3d(10, paths, obs_nodes=obs)

In [ ]:
print("len(paths) =", len(paths))

In [ ]:
to[408]

In [ ]:
to[398]

In [ ]:
vis_gridpath_3d(5)

In [ ]:
to_without_obs[23]

In [ ]:
paths